In [ ]:
pip install split-folders

In [ ]:
import zipfile
# import splitfolders
import os
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Extract the archive
local_zip = '/content/drive/MyDrive/dataset_capstone/224x224_5class.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [ ]:
# delete class
!rm -rf '/content/data/test/Oblong'
!rm -rf '/content/data/test/Oval'
!rm -rf '/content/data/train/Oblong'
!rm -rf '/content/data/train/Oval'

In [ ]:
# base_dir = 'data/dataset_capstone/'
# splitfolders.ratio(base_dir, output = 'data/dataset_capstone/dataset', seed = 1337, ratio=(0.8, 0.2), group_prefix=None)
# train_dir = 'data/dataset_capstone/dataset/train'
# val_dir = 'data/dataset_capstone/dataset/val'

In [ ]:
!unzip /content/drive/MyDrive/dataset_capstone/224x224_3class.zip -d /content

Output streaming akan dipotong hingga 5000 baris terakhir.
  inflating: /content/train/Heart/Heart__0_2550.jpg  
  inflating: /content/train/Heart/Heart__0_1040.jpg  
  inflating: /content/train/Heart/Heart__0_1313.jpg  
  inflating: /content/train/Heart/Heart__0_6892.jpg  
  inflating: /content/train/Heart/Heart__0_5888.jpg  
  inflating: /content/train/Heart/Heart__0_3340.jpg  
  inflating: /content/train/Heart/Heart__0_6938.jpg  
  inflating: /content/train/Heart/Heart__0_5747.jpg  
  inflating: /content/train/Heart/Heart__0_3033.jpg  
  inflating: /content/train/Heart/Heart__0_5149.jpg  
  inflating: /content/train/Heart/Heart__0_5451.jpg  
  inflating: /content/train/Heart/Heart__0_2823.jpg  
  inflating: /content/train/Heart/Heart__0_4741.jpg  
  inflating: /content/train/Heart/Heart__0_376.jpg  
  inflating: /content/train/Heart/Heart__0_9347.jpg  
  inflating: /content/train/Heart/Heart__0_1161.jpg  
  inflating: /content/train/Heart/Heart__0_7757.jpg  
  inflating: /content/tr

In [ ]:
!mkdir /content/temp
!mkdir /content/temp2

In [ ]:
!mv /content/train/{Oblong,Oval} /content/temp2
!mv /content/test/{Oblong,Oval} /content/temp2

mv: cannot stat '/content/train/Oblong': No such file or directory
mv: cannot stat '/content/train/Oval': No such file or directory


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=10,
      # width_shift_range=0.3,
      # height_shift_range=0.3,
      # shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='reflect')

validation_datagen = ImageDataGenerator(
                    rescale=1./255)

In [ ]:
train_dir = '/content/train'
val_dir = '/content/test'

In [ ]:
!ls -1 /content/train/Heart | wc -l
!ls -1 /content/test/Heart | wc -l
!ls -1 /content/train/Round | wc -l
!ls -1 /content/test/Round | wc -l
!ls -1 /content/train/Square | wc -l
!ls -1 /content/test/Square | wc -l

2124
588
2113
587
2145
581


In [ ]:
train_generator = training_datagen.flow_from_directory(
	train_dir,
	target_size=(224,224),
	class_mode='sparse',
  batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	val_dir,
	target_size=(224,224),
	class_mode='sparse',
  batch_size=32
)

Found 6382 images belonging to 3 classes.
Found 1756 images belonging to 3 classes.


In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    # regulizer
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(3, activation='softmax')
])


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/dataset_capstone/saved_model_Dec_17_2023_nv4')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
# model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), metrics=['accuracy'])

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Train the model
history = model.fit(train_generator, epochs=25, validation_data = validation_generator, verbose = 1)

Epoch 1/25
200/200 [==============================] - 96s 438ms/step - loss: 0.9035 - accuracy: 0.7678 - val_loss: 1.2186 - val_accuracy: 0.6298
Epoch 2/25
200/200 [==============================] - 87s 433ms/step - loss: 0.8894 - accuracy: 0.7775 - val_loss: 1.2142 - val_accuracy: 0.6276
Epoch 3/25
200/200 [==============================] - 87s 435ms/step - loss: 0.8801 - accuracy: 0.7800 - val_loss: 1.2124 - val_accuracy: 0.6372
Epoch 4/25
200/200 [==============================] - 88s 440ms/step - loss: 0.8856 - accuracy: 0.7852 - val_loss: 1.2384 - val_accuracy: 0.6253
Epoch 5/25
200/200 [==============================] - 84s 422ms/step - loss: 0.8880 - accuracy: 0.7871 - val_loss: 1.2613 - val_accuracy: 0.6167
Epoch 6/25
200/200 [==============================] - 87s 433ms/step - loss: 0.8844 - accuracy: 0.7911 - val_loss: 1.2833 - val_accuracy: 0.6213
Epoch 7/25
200/200 [==============================] - 87s 436ms/step - loss: 0.8815 - accuracy: 0.8027 - val_loss: 1.2855 - val_ac

In [ ]:
tf.keras.saving.save_model(model, '/content/drive/MyDrive/dataset_capstone/saved_model_Dec_17_2023_nv5')

In [ ]:
pip install tensorflowjs

In [ ]:
import tensorflowjs

In [ ]:
import time
saved_model_path = "./{}.h5".format(int(time.time()))

model.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

In [ ]:
saved_model_path = "./{}.h5".format(int(time.time()))

history.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import os
import random

# Load your saved model
# model_path = 'path/to/your/saved_model'
loaded_model =  model #tf.keras.models.load_model(model_path)

# Specify the directory containing your test images
# test_directory = '/content/train/Heart'
# test_directory = '/content/train/Round'
test_directory = '/content/train/Square'

# Get a list of image files in the test directory
image_files = [f for f in os.listdir(test_directory) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Randomly select 10 samples
random_10_samples = random.sample(image_files, 10)

# Class labels corresponding to Heart, Round, Square
class_labels = ['Heart', 'Round', 'Square']

# Iterate over the 10 randomly selected image files and make predictions
for img_file in random_10_samples:
    img_path = os.path.join(test_directory, img_file)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Make a prediction
    predictions = loaded_model.predict(img_array)

    # Get the predicted class index (0, 1, or 2)
    predicted_class_index = np.argmax(predictions)

    # Get the corresponding class label
    predicted_class_label = class_labels[predicted_class_index]

    # Print the prediction result
    print(f"\nPredicted face shape for {img_file}: {predicted_class_label}")


1/1 [==============================] - 0s 27ms/step

Predicted face shape for Square__0_6374.jpg: Square
1/1 [==============================] - 0s 25ms/step

Predicted face shape for Square__0_6094.jpg: Heart
1/1 [==============================] - 0s 27ms/step

Predicted face shape for Square__0_6923.jpg: Heart
1/1 [==============================] - 0s 27ms/step

Predicted face shape for Square__0_5631.jpg: Square
1/1 [==============================] - 0s 26ms/step

Predicted face shape for Square__0_6091.jpg: Round
1/1 [==============================] - 0s 33ms/step

Predicted face shape for Square__0_707.jpg: Square
1/1 [==============================] - 0s 25ms/step

Predicted face shape for Square__0_1762.jpg: Square
1/1 [==============================] - 0s 26ms/step

Predicted face shape for Square__0_2566.jpg: Square
1/1 [==============================] - 0s 31ms/step

Predicted face shape for Square__0_2479.jpg: Round
1/1 [==============================] - 0s 27ms/step

Predict

In [ ]:
import os

def count_files_in_directory(directory_path):
    # Get the list of files in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

    # Count the number of files
    num_files = len(files)

    return num_files

# Specify the directory path
directory_path = '/content/test/Heart'

# Call the function to count files
num_files = count_files_in_directory(directory_path)

# Print the result
print(f"Number of files in the directory: {num_files}")


Number of files in the directory: 588


In [ ]:
!zip /content/model.zip -r /content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5

  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/ (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/variables/ (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/variables/variables.data-00000-of-00001 (deflated 19%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/variables/variables.index (deflated 65%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/assets/ (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/saved_model.pb (deflated 87%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/fingerprint.pb (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/keras_metadata.pb (deflated 92%)


In [ ]:
!cp /content/model.zip /content/drive/MyDrive/style-mate/